In [ ]:
import pandas as pd 
import numpy as np 
import requests 
import time 
import bs4
from bs4 import BeautifulSoup
import random
import re
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import sys

In [ ]:
#UPDATE TOKEN
session_id = '133-4548745-1768158'
tolkien = 'gC89ZUO4Zdyha7Y9n5OMwMnKuNfvFJammpPdqREAAAAMAAAAAGLh0UlyYXcAAAAA;hBDEV5Vinnug5n0vW8BdpKDY9jttDR24eLsOohiOmOJUAAAAAGLh0UkAAAAB'
#AMAZON SEARCH THAT SHOWS PRODUCT ON FIRST PAGE
search1 = 'black and decker air fryer'
#EXACT PRODUCT NAME
productID = 'BLACK+DECKER Purify 2-Liter Air Fryer'


#labels (internal)
caregory = 'air fryer'
key_word = 'power'

#positive vs negative reviews (scraping 5 star + 1 star reviews for the given product)
reviewhigh = 'five'
rh = '5'
reviewlow = 'one'
rl = '1'
pages = 3


In [ ]:
search = search1.lower().replace(' ','+')
productID = productID.lower()
lenPID = int(len(productID))
url = 'https://www.amazon.com/s?k='+search+':&page='
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}
cookies = {'session-id':session_id,'session-id-time':'2082787201l','i18n-prefs':'USD','csm-hit':'tb:s-1KBXPV9W0G7AT013EMW9|1648478470844&t:1648478472672&adb:adblk_no','ubid-main':'132-0411939-8328406','session-token':tolkien}
html = requests.get(url,headers = headers, cookies =cookies).text
soup = bs4.BeautifulSoup(html, "html.parser")
xc = str(soup).split('"a-size-medium a-color-base a-text-normal">')
for xx in range(len(xc)):
  if xc[xx][:lenPID].lower() == productID:
    try:
      price = xc[xx].split('<span class="a-offscreen">')[1].split('<')[0].replace('$','')
      product = xc[xx].split('<')[0]
      reviewAvg = xc[xx].split('<div class="a-row a-size-small"><span aria-label="')[1].split('">')[0].split()[0]
      reviewCount = xc[xx].split('</span><span aria-label="')[1].split('"')[0]
      link = 'https://www.amazon.com' + xc[xx-1].split('href="')[-1].split('"')[0]
    except:
      fdf = 0
  else:
    fdf = 0

pid = link.split('dp/')[1].split('/')[0]
hightext = []
lowtext = []
qtext = []

for ux in range(pages):
  high_review_url = 'https://www.amazon.com/product-reviews/'+pid+'/ref=acr_dp_hist_'+rh+'?ie=UTF8&filterByStar='+reviewhigh+'_star&reviewerType=all_reviews&pageNumber='+str(ux)+'#reviews-filter-bar'
  low_review_url = 'https://www.amazon.com/product-reviews/'+pid+'/ref=acr_dp_hist_'+rl+'?ie=UTF8&filterByStar='+reviewlow+'_star&reviewerType=all_reviews&pageNumber='+str(ux)+'#reviews-filter-bar'
  question_url = 'https://www.amazon.com/ask/questions/asin/'+pid+'/'+str(ux)+'/ref=ask_dp_iaw_ql_hza?'
  url = high_review_url
  ht = []
  html = requests.get(url,headers = headers, cookies =cookies).text
  soup = bs4.BeautifulSoup(html, "html.parser")
  xc = str(soup).split('Reviewed in the United States')
  for xi in range(len(xc)-1):
    try:
      ht.append(xc[xi].split('<span>')[1].split('</span>')[0])
    except:
      fdf = 0
  for xu in range(len(ht)):
    if int(len(ht[xu].split('{"')))>=2:
      fdf = 0
    else:
      hightext.append(ht[xu].replace("\'","").replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
  time.sleep(random.uniform(0,9))
  url = low_review_url
  lt = []
  html = requests.get(url,headers = headers, cookies =cookies).text
  soup = bs4.BeautifulSoup(html, "html.parser")
  xc = str(soup).split('Reviewed in the United States')
  for xi in range(len(xc)-1):
    try:
      lt.append(xc[xi].split('<span>')[1].split('</span>')[0])
    except:
      fdf = 0
  for xu in range(len(lt)):
    if int(len(lt[xu].split('{"')))>=2:
      fdf = 0
    else:
      lowtext.append(lt[xu].replace("\'","").replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
  time.sleep(random.uniform(0,9))
  url = question_url
  html = requests.get(url,headers = headers, cookies =cookies).text
  soup = bs4.BeautifulSoup(html, "html.parser")
  xc = str(soup).split('Question:')
  qt=[]
  for xj in range(len(xc)):
    qt.append(xc[xj].split('type="widget">')[1].split('</')[0].replace('\n                ','').replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
  for xw in range(len(qt)):
    if int(len(str(qt[xw]).split('<')))>=2:
      fdf = 0
    else:
      qtext.append(qt[xw].replace('  ','').replace('\n',''))
  time.sleep(random.uniform(0,9))

df = {'product':[],'reviewAvg':[],'reviewCount':[],'price':[],'link':[],'positive_text':[],'negative_text':[],'question_text':[]}
df['product'].append(product)
df['reviewAvg'].append(reviewAvg)
df['reviewCount'].append(reviewCount)
df['price'].append(price)
df['link'].append(link)
df['positive_text'].append(hightext)
df['negative_text'].append(lowtext)
df['question_text'].append(qtext)
pd.DataFrame(df)

,product,reviewAvg,reviewCount,price,link,positive_text,negative_text,question_text
0,"BLACK+DECKER Purify 2-Liter Air Fryer, Black/S...",4.5,"2,003",78.27,https://www.amazon.com/BLACK-DECKER-2-Liter-St...,"[Prime, Listen to only this review. Forget eve...","[Prime, My food came out pretty good with very...","[Do you have to use oil ?, can i put metal/sta..."
